SET UP môi trường nhé hẹ hẹ

In [1]:
!pip install sentence-transformers faiss-cpu networkx pandas numpy


In [2]:
!pip -q install tiktoken

import tiktoken

LLM extractor (JSON schema)

In [8]:
import os

# set key ở đây (đừng hardcode trong code dài)
os.environ["NVAPI_KEY"] = "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"  # <-- điền key của bạn

JSON_PATH = "thongtu80.json"
CACHE_DIR = "./tt80_cache_full"   # thư mục lưu mọi thứ


In [4]:
!pip install openai

In [21]:
import os, json, re, logging
from datetime import datetime
from typing import Any, Dict, List, Optional, Set, Tuple

import numpy as np
import networkx as nx
import faiss
from sentence_transformers import SentenceTransformer

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("OFFLINE_BUILD_NO_LLM")

# config
# JSON_PATH  = "thongtu80.json"
JSON_FILES = [
    "thongtu80.json",
    "thongtu78.json",
    "nghidinh123.json",
    "nghidinh125.json",
    "GTGT2008.json",
    "QLT2019.json"
]

CACHE_DIR  = "./tt80_artifacts_no_llm"

EMBED_MODEL = "intfloat/multilingual-e5-base"
BATCH_EMBED = 32

CHUNK_TOKENS = 800
CHUNK_OVERLAP = 80
CHUNK_CHAR = 1200
CHUNK_CHAR_OVERLAP = 120

# KG params (offline)
TOP_K_TERMS_PER_CHUNK = 18
MIN_TERM_LEN = 3
MAX_TERM_WORDS = 6
COOC_WINDOW = 2  # connect term i with i+1..i+COOC_WINDOW in the same chunk

# phần này là để xử lý JSON văn bản pháp luật
def norm(s: Optional[str]) -> str:
    return (s or "").replace("\r", "").strip()

def join_path(parts: List[str]) -> str:
    return " > ".join([p for p in parts if p])

def flatten_legal_tree(node: Dict[str, Any], path: List[str], out: List[Dict[str, Any]]):
    ntype = node.get("type", "")
    title = norm(node.get("title"))
    content = norm(node.get("content"))
    new_path = path + ([title] if title else [])
    if content:
        out.append({"type": ntype, "path": join_path(new_path), "text": content})
    for ch in node.get("children", []) or []:
        flatten_legal_tree(ch, new_path, out)

def load_legal_json(json_path: str) -> Dict[str, Any]:
    with open(json_path, "r", encoding="utf-8") as f:
        return json.load(f)

def flatten_to_passages(doc: Dict[str, Any], source_file: str = "") -> List[str]:
    """Flatten 1 document thành passages, thêm source_file vào metadata"""
    info = doc.get("document_info", {})
    base = [
        f"Văn bản: {norm(info.get('title'))}",
    ]
    if source_file:
        base.append(f"Nguồn: {source_file}")
    base.extend([
        f"URL: {norm(info.get('url'))}",
        f"Crawled: {norm(info.get('crawled_at'))}",
    ])
    
    units: List[Dict[str, Any]] = []
    for top in doc.get("body", []) or []:
        flatten_legal_tree(top, base, units)

    passages = []
    for u in units:
        header = f"[{u['type']}] {u['path']}"
        passages.append(header + "\n" + u["text"])
    return passages

# ============================================================
# LOAD MULTIPLE JSON FILES
# ============================================================

def load_all_json_files(json_files: List[str]) -> Tuple[List[str], Dict[str, Any]]:
    """
    Load tất cả JSON files và trả về:
    - all_passages: list tất cả passages từ mọi file
    - metadata: thông tin về số lượng passages từ mỗi file
    """
    all_passages = []
    metadata = {
        "files": [],
        "passages_per_file": {}
    }
    
    for json_path in json_files:
        if not os.path.exists(json_path):
            logger.warning(f"⚠️ File not found: {json_path}, skipping...")
            continue
        
        logger.info(f"📄 Loading: {json_path}")
        try:
            doc = load_legal_json(json_path)
            passages = flatten_to_passages(doc, source_file=json_path)
            
            metadata["files"].append(json_path)
            metadata["passages_per_file"][json_path] = len(passages)
            
            all_passages.extend(passages)
            logger.info(f"   → {len(passages)} passages")
            
        except Exception as e:
            logger.error(f"❌ Error loading {json_path}: {e}")
            continue
    
    logger.info(f"✅ Total: {len(all_passages)} passages from {len(metadata['files'])} files")
    return all_passages, metadata


# Chunking nhé ae
def try_import_tiktoken():
    try:
        import tiktoken
        return tiktoken
    except Exception:
        return None

def token_chunk(text: str, chunk_tokens: int, overlap: int, tokenizer_name="cl100k_base") -> List[str]:
    tiktoken = try_import_tiktoken()
    if tiktoken is None:
        raise ModuleNotFoundError("tiktoken not installed")
    enc = tiktoken.get_encoding(tokenizer_name)
    toks = enc.encode(text)
    step = max(1, chunk_tokens - overlap)
    out = []
    for i in range(0, len(toks), step):
        sub = toks[i:i+chunk_tokens]
        if sub:
            out.append(enc.decode(sub))
    return out

def char_chunk(text: str, chunk_size: int, overlap: int) -> List[str]:
    seps = ["\n\n", "\n", ". ", "; ", ", ", " ", ""]
    chunks: List[str] = []
    buf = text.strip()
    if not buf:
        return []
    start = 0
    while start < len(buf):
        end = min(len(buf), start + chunk_size)
        piece = buf[start:end]
        cut = -1
        for sep in seps:
            idx = piece.rfind(sep)
            if idx > cut:
                cut = idx
        if cut <= 0:
            cut = len(piece)
        else:
            cut = cut + 1
        chunk = piece[:cut].strip()
        if chunk:
            chunks.append(chunk)
        start = start + max(1, cut - overlap)
    return chunks

def make_chunks(passages: List[str]) -> List[str]:
    dataset: List[str] = []
    use_token = try_import_tiktoken() is not None
    logger.info(f"Chunking mode: {'tiktoken' if use_token else 'char-fallback'}")
    for p in passages:
        if use_token:
            try:
                dataset.extend(token_chunk(p, CHUNK_TOKENS, CHUNK_OVERLAP))
            except Exception:
                dataset.extend(char_chunk(p, CHUNK_CHAR, CHUNK_CHAR_OVERLAP))
        else:
            dataset.extend(char_chunk(p, CHUNK_CHAR, CHUNK_CHAR_OVERLAP))
    return dataset

# ==============
# Embedding + FAISS
# ==============
embedder = SentenceTransformer(EMBED_MODEL)

def embed_texts(texts: List[str], batch_size=32) -> np.ndarray:
    vecs = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inp = [f"passage: {t}" for t in batch]
        v = embedder.encode(inp, normalize_embeddings=True)
        vecs.append(np.asarray(v, dtype="float32"))
    return np.vstack(vecs)

def build_faiss(embeddings: np.ndarray) -> faiss.Index:
    d = embeddings.shape[1]
    idx = faiss.IndexFlatIP(d)
    idx.add(embeddings.astype("float32"))
    return idx

# ==============
# OFFLINE Entity / Term extraction (NO LLM)
# ==============
def normalize_term(s: str) -> str:
    s = s.strip()
    s = re.sub(r"\s+", " ", s)
    # remove surrounding quotes
    s = s.strip("“”\"' ")
    return s

def extract_quoted_terms(text: str) -> List[str]:
    # captures “...”, "...", '...'
    terms = []
    for a,b,c in re.findall(r"“([^”]{2,120})”|\"([^\"]{2,120})\"|'([^']{2,120})'", text):
        t = normalize_term(a or b or c)
        if t:
            terms.append(t)
    return terms

def extract_legal_markers(text: str) -> List[str]:
    # keep lawy concepts + headings
    # examples: Điều 1, Khoản 2, điểm a, Nghị định số 126/2020/NĐ-CP
    pats = [
        r"(Điều\s+\d+[A-Za-z]?)",
        r"(Khoản\s+\d+[A-Za-z]?)",
        r"(điểm\s+[a-zđ])",
        r"(Nghị\s+định\s+số\s+\d+\/\d+\/[A-ZĐ\-]+)",
        r"(Thông\s+tư\s+\d+\/\d+\/[A-ZĐ\-]+)",
        r"(Luật\s+[A-ZÀ-Ỹa-zà-ỹ\s]{3,80})",
    ]
    out = []
    for pat in pats:
        out.extend([normalize_term(x) for x in re.findall(pat, text, flags=re.I)])
    return out

def extract_terms_yake(text: str, top_k: int) -> List[str]:
    try:
        import yake
        kw_extractor = yake.KeywordExtractor(lan="vi", n=MAX_TERM_WORDS, top=top_k)
        kws = kw_extractor.extract_keywords(text)
        # yake returns list[(kw, score)] lower score better
        kws = sorted(kws, key=lambda x: x[1])[:top_k]
        return [normalize_term(k) for k,_ in kws if k]
    except Exception:
        return []

def extract_terms_underthesea(text: str, top_k: int) -> List[str]:
    # optional: better Vietnamese word segmentation + noun phrases
    try:
        from underthesea import pos_tag
        tags = pos_tag(text)
        # collect sequences of N / NP / A (rough)
        terms = []
        buf = []
        for w,t in tags:
            if t in ("N", "Np", "Ny", "Nc", "A"):
                buf.append(w)
            else:
                if 1 <= len(buf) <= MAX_TERM_WORDS:
                    terms.append(normalize_term(" ".join(buf)))
                buf = []
        if 1 <= len(buf) <= MAX_TERM_WORDS:
            terms.append(normalize_term(" ".join(buf)))
        # filter length
        terms = [x for x in terms if len(x) >= MIN_TERM_LEN]
        # unique preserve order
        seen = set()
        out = []
        for x in terms:
            lx = x.lower()
            if lx not in seen:
                seen.add(lx)
                out.append(x)
        return out[:top_k]
    except Exception:
        return []

def extract_terms_offline(text: str, top_k: int = TOP_K_TERMS_PER_CHUNK) -> List[str]:
    # Priority order: quoted terms + legal markers + underthesea + yake + fallback regex ngrams
    terms = []
    terms.extend(extract_quoted_terms(text))
    terms.extend(extract_legal_markers(text))

    # underthesea noun-ish phrases
    ut = extract_terms_underthesea(text, top_k=top_k)
    if ut:
        terms.extend(ut)

    # yake keyphrases
    yk = extract_terms_yake(text, top_k=top_k)
    if yk:
        terms.extend(yk)

    # fallback: common patterns like "cơ quan thuế", "người nộp thuế" by scanning
    # pick 2-5 word grams around these trigger words
    triggers = ["thuế", "hồ sơ", "nghĩa vụ", "quy định", "cơ quan", "người", "thương mại", "nền tảng", "hoàn"]
    words = re.findall(r"[A-Za-zÀ-ỹ0-9_]+", text.lower())
    for i,w in enumerate(words):
        if w in triggers:
            for n in (2,3,4):
                gram = " ".join(words[i:i+n])
                gram = normalize_term(gram)
                if len(gram) >= MIN_TERM_LEN:
                    terms.append(gram)

    # clean + unique
    clean = []
    seen = set()
    for t in terms:
        t = normalize_term(t)
        if not t:
            continue
        if len(t) < MIN_TERM_LEN:
            continue
        if len(t.split()) > MAX_TERM_WORDS:
            continue
        key = t.lower()
        if key not in seen:
            seen.add(key)
            clean.append(t)

    return clean[:top_k]

# ==============
# Build KG (NO LLM): co-occurrence graph
# ==============
def build_kg_and_chunk_entities_no_llm(dataset: List[str]) -> Tuple[nx.DiGraph, List[Set[str]]]:
    kg = nx.DiGraph()
    chunk_entities: List[Set[str]] = []

    for i, chunk in enumerate(dataset):
        terms = extract_terms_offline(chunk, top_k=TOP_K_TERMS_PER_CHUNK)
        ent_set = set(terms)
        chunk_entities.append(ent_set)

        for t in ent_set:
            if not kg.has_node(t):
                kg.add_node(t)

        # co-occurrence edges (ordered terms)
        ordered = terms
        for a_idx, a in enumerate(ordered):
            for b_idx in range(a_idx+1, min(len(ordered), a_idx+1+COOC_WINDOW)):
                b = ordered[b_idx]
                if a == b:
                    continue
                # directional edge with relation label
                if kg.has_edge(a, b):
                    kg[a][b]["weight"] = kg[a][b].get("weight", 1) + 1
                else:
                    kg.add_edge(a, b, relation="co_occurs_with", weight=1, source_chunk=i)

        if (i+1) % 200 == 0:
            logger.info(f"KG progress {i+1}/{len(dataset)} | nodes={kg.number_of_nodes()} edges={kg.number_of_edges()}")

    return kg, chunk_entities

# ====== RUN OFFLINE BUILD ======
os.makedirs(CACHE_DIR, exist_ok=True)

# logger.info("Load JSON -> passages")
# doc = load_legal_json(JSON_PATH)
# passages = flatten_to_passages(doc)

# os.makedirs(CACHE_DIR, exist_ok=True)

# Load tất cả JSON files
logger.info("="*60)
logger.info("LOADING MULTIPLE JSON FILES")
logger.info("="*60)
passages, file_metadata = load_all_json_files(JSON_FILES)

if not passages:
    raise ValueError("❌ No passages loaded! Check JSON_FILES paths.")

logger.info("\nChunking")
dataset = make_chunks(passages)
logger.info(f"Total chunks = {len(dataset)}")

logger.info("\nEmbedding + FAISS")
embeddings = embed_texts(dataset, batch_size=BATCH_EMBED)
index = build_faiss(embeddings)
logger.info(f"FAISS ntotal = {index.ntotal}")

logger.info("\nBuild KG + chunk_entities (NO LLM)")
kg, chunk_entities = build_kg_and_chunk_entities_no_llm(dataset)

# ====== SAVE ARTIFACTS ======
with open(os.path.join(CACHE_DIR, "chunks.json"), "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False)

faiss.write_index(index, os.path.join(CACHE_DIR, "faiss.index"))
np.save(os.path.join(CACHE_DIR, "embeddings.npy"), embeddings)

import pickle
with open(os.path.join(CACHE_DIR, "kg.pkl"), "wb") as f:
    pickle.dump(kg, f)

with open(os.path.join(CACHE_DIR, "chunk_entities.json"), "w", encoding="utf-8") as f:
    json.dump([list(s) for s in chunk_entities], f, ensure_ascii=False)

meta = {
    "created_at": datetime.now().isoformat(),
    "embedding_model": EMBED_MODEL,
    "num_chunks": len(dataset),
    "kg_nodes": kg.number_of_nodes(),
    "kg_edges": kg.number_of_edges(),
    "build_mode": "NO_LLM",
    "source_files": file_metadata["files"],
    "passages_per_file": file_metadata["passages_per_file"],
}
with open(os.path.join(CACHE_DIR, "meta.json"), "w", encoding="utf-8") as f:
    json.dump(meta, f, ensure_ascii=False, indent=2)

logger.info(f"\n{'='*60}")
logger.info(f"✅ DONE. Saved artifacts to: {CACHE_DIR}")
logger.info(f"   Files processed: {len(file_metadata['files'])}")
logger.info(f"   Total chunks: {len(dataset)}")
logger.info(f"   KG nodes: {kg.number_of_nodes()}")
logger.info(f"   KG edges: {kg.number_of_edges()}")
logger.info(f"{'='*60}")


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: intfloat/multilingual-e5-base
INFO:httpx:HTTP Request: HEAD https://huggingface.co/intfloat/multilingual-e5-base/resolve/main/modules.json "HTTP/1.1 307 Temporary Redirect"
INFO:httpx:HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/intfloat/multilingual-e5-base/835193815a3936a24a0ee7dc9e3d48c1fbb19c55/modules.json "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD https://huggingface.co/intfloat/multilingual-e5-base/resolve/main/config_sentence_transformers.json "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: HEAD https://huggingface.co/intfloat/multilingual-e5-base/resolve/main/config_sentence_transformers.json "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: HEAD https://huggingface.co/intfloat/multilingual-e5-base/resolve/main/README.md "HTTP/1.1 307 Temporary Redirect"
INFO:httpx:HTTP Request: HEAD https:/

In [28]:
import os, json, re
import numpy as np
import networkx as nx
import faiss
from sentence_transformers import SentenceTransformer
from openai import OpenAI
from typing import List, Set

CACHE_DIR = "./tt80_artifacts_no_llm"
MODEL_ANSWER = "moonshotai/kimi-k2-instruct-0905"
TOP_K = 6
ALPHA = 0.85

# --- FORCE RELOAD: đặt thành True khi muốn reload artifacts mới ---
FORCE_RELOAD = True

# --- load artifacts ---
if FORCE_RELOAD or "BOT_READY" not in globals():
    print("🔄 Loading artifacts...")
    
    with open(os.path.join(CACHE_DIR, "meta.json"), "r", encoding="utf-8") as f:
        META = json.load(f)

    with open(os.path.join(CACHE_DIR, "chunks.json"), "r", encoding="utf-8") as f:
        DATASET = json.load(f)

    INDEX = faiss.read_index(os.path.join(CACHE_DIR, "faiss.index"))
    
    import pickle
    with open(os.path.join(CACHE_DIR, "kg.pkl"), "rb") as f:
        KG = pickle.load(f)

    with open(os.path.join(CACHE_DIR, "chunk_entities.json"), "r", encoding="utf-8") as f:
        CHUNK_ENTITIES = [set(x) for x in json.load(f)]

    EMBEDDER = SentenceTransformer(META["embedding_model"])
    BOT_READY = True

    print("✅ Artifacts loaded")
    print(f"   Chunks: {len(DATASET)}")
    print(f"   KG nodes: {KG.number_of_nodes()} | KG edges: {KG.number_of_edges()}")
    print(f"   Source files: {META.get('source_files', ['unknown'])}")

# --- LLM client ---
if not os.getenv("NVAPI_KEY"):
    raise RuntimeError("Missing NVAPI_KEY env var.")
client = OpenAI(base_url="https://integrate.api.nvidia.com/v1", api_key=os.getenv("NVAPI_KEY"))

def llm_answer(prompt: str, temperature=0.2, max_tokens=450) -> str:
    resp = client.chat.completions.create(
        model=MODEL_ANSWER,
        messages=[{"role":"user","content":prompt}],
        temperature=temperature,
        top_p=0.9,
        max_tokens=max_tokens
    )
    return resp.choices[0].message.content.strip()

# --- fast keyword/entity extraction without LLM ---
STOP = set(["là", "và", "của", "theo", "đối", "với", "như", "thế", "nào", "gì", "bao", "gồm", "quy", "định", "hướng", "dẫn", "không", "có", "một", "các", "được"])

def fast_terms(query: str) -> Set[str]:
    q = query.lower()
    quoted = re.findall(r"“([^”]+)”|\"([^\"]+)\"", query)
    terms = set()
    for a,b in quoted:
        s = (a or b).strip().lower()
        if s:
            terms.add(s)
    words = re.findall(r"[a-zA-ZÀ-ỹ0-9_]+", q)
    for w in words:
        if len(w) >= 3 and w not in STOP:
            terms.add(w)
    return terms

def semantic_search(query: str, top_k: int):
    q_vec = EMBEDDER.encode([f"query: {query}"], normalize_embeddings=True).astype("float32")
    D, I = INDEX.search(q_vec, top_k)
    return [(int(I[0][j]), float(D[0][j])) for j in range(top_k)]

def graph_scores(query_terms: Set[str]) -> np.ndarray:
    scores = np.zeros(len(DATASET), dtype=float)
    if not query_terms:
        return scores
    for i, ents in enumerate(CHUNK_ENTITIES):
        hit = 0
        for t in query_terms:
            for e in ents:
                if t in e.lower():
                    hit += 1
                    break
        scores[i] = hit
    if scores.max() > 0:
        scores = scores / (scores.max() + 1e-12)
    return scores

def kg_facts_from_terms(query_terms: Set[str], limit=10) -> List[str]:
    facts = []
    for t in list(query_terms)[:8]:
        cand = [n for n in KG.nodes() if t in str(n).lower()]
        for node in cand[:2]:
            for nb in list(KG.successors(node))[:3]:
                rel = KG.get_edge_data(node, nb).get("relation", "related_to")
                facts.append(f"{node} --{rel}--> {nb}")
                if len(facts) >= limit:
                    return facts
    return facts

def retrieve_context(query: str, top_k=TOP_K, alpha=ALPHA):
    sem = semantic_search(query, top_k=top_k*2)
    terms = fast_terms(query)
    gs = graph_scores(terms)

    combined = []
    for idx, sem_score in sem:
        g = float(gs[idx])
        combined.append((idx, alpha*sem_score + (1-alpha)*g))
    combined.sort(key=lambda x: x[1], reverse=True)
    idxs = [i for i,_ in combined[:top_k]]

    ctx = "\n\n".join([DATASET[i] for i in idxs])
    facts = kg_facts_from_terms(terms, limit=10)
    if facts:
        ctx += "\n\nKG_FACTS:\n" + "\n".join(facts)
    return ctx, idxs

def ask(query: str):
    ctx, idxs = retrieve_context(query)
    prompt = f"""
Bạn là trợ lý hỏi đáp luật Việt Nam.

sử dụng những kiến thức từ knowledge graph, không tự bịa đặt nhé

hơn nữa, những kiến thức có trong CONTEXT rất quan trọng, phải dùng đến chúng để trả lời câu hỏi. Nên hãy dùng hết để tóm tắt để trả lời

CHỈ trả lời dựa trên CONTEXT. Không bịa.
Nếu CONTEXT không có thông tin, trả lời đúng 1 dòng:
no relevant information found

CONTEXT:
{ctx}

QUESTION:
{query}

ANSWER:
"""
    ans = llm_answer(prompt, temperature=0.2, max_tokens=450)
    return ans, idxs

print("\n✅ Chatbot mode ready.")

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: intfloat/multilingual-e5-base


🔄 Loading artifacts...


INFO:httpx:HTTP Request: HEAD https://huggingface.co/intfloat/multilingual-e5-base/resolve/main/modules.json "HTTP/1.1 307 Temporary Redirect"
INFO:httpx:HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/intfloat/multilingual-e5-base/835193815a3936a24a0ee7dc9e3d48c1fbb19c55/modules.json "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD https://huggingface.co/intfloat/multilingual-e5-base/resolve/main/config_sentence_transformers.json "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: HEAD https://huggingface.co/intfloat/multilingual-e5-base/resolve/main/config_sentence_transformers.json "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: HEAD https://huggingface.co/intfloat/multilingual-e5-base/resolve/main/README.md "HTTP/1.1 307 Temporary Redirect"
INFO:httpx:HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/intfloat/multilingual-e5-base/835193815a3936a24a0ee7dc9e3d48c1fbb19c55/README.md "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD https://huggingface.

✅ Artifacts loaded
   Chunks: 3488
   KG nodes: 4724 | KG edges: 18778
   Source files: ['thongtu80.json', 'thongtu78.json', 'nghidinh123.json', 'nghidinh125.json', 'GTGT2008.json', 'QLT2019.json']

✅ Chatbot mode ready.


In [33]:
# Cell 9: GraphRAG Engine v2 - Full Implementation

from typing import Dict, List, Set, Tuple

# ============================================================
# GRAPHRAG ENGINE V2 - với KG Expansion cho Retrieval
# ============================================================

def get_kg_expanded_terms(seed_terms: Set[str], hops: int = 2, max_neighbors_per_node: int = 5) -> Dict[str, float]:
    """
    Từ seed_terms, đi qua KG để lấy thêm related terms với decay weight.
    Returns: {term: weight} — weight càng cao = càng relevant
    """
    expanded = {}
    
    # seed terms có weight = 1.0
    for t in seed_terms:
        expanded[t.lower()] = 1.0
    
    # BFS expansion với decay
    frontier = list(seed_terms)
    visited = set(t.lower() for t in seed_terms)
    
    for hop in range(1, hops + 1):
        decay = 1.0 / (hop + 1)  # hop 1 → 0.5, hop 2 → 0.33
        next_frontier = []
        
        for term in frontier:
            # tìm nodes trong KG match term
            matching_nodes = [n for n in KG.nodes() if term.lower() in str(n).lower()]
            
            for node in matching_nodes[:3]:  # giới hạn nodes per term
                # lấy neighbors (cả successors và predecessors)
                neighbors = list(KG.successors(node)) + list(KG.predecessors(node))
                
                # sort by edge weight nếu có
                weighted_neighbors = []
                for nb in neighbors:
                    edge_data = KG.get_edge_data(node, nb) or KG.get_edge_data(nb, node) or {}
                    w = edge_data.get("weight", 1)
                    weighted_neighbors.append((nb, w))
                
                weighted_neighbors.sort(key=lambda x: x[1], reverse=True)
                
                for nb, edge_weight in weighted_neighbors[:max_neighbors_per_node]:
                    nb_lower = str(nb).lower()
                    if nb_lower not in visited:
                        visited.add(nb_lower)
                        next_frontier.append(str(nb))
                        # weight = decay * normalized edge weight
                        score = decay * min(edge_weight / 5.0, 1.0)
                        expanded[nb_lower] = max(expanded.get(nb_lower, 0), score)
        
        frontier = next_frontier
    
    return expanded


def graph_scores_v2(expanded_terms: Dict[str, float]) -> np.ndarray:
    """
    Chấm điểm chunks dựa trên expanded terms từ KG traversal.
    Weighted matching thay vì binary.
    """
    scores = np.zeros(len(DATASET), dtype=float)
    if not expanded_terms:
        return scores
    
    for i, ents in enumerate(CHUNK_ENTITIES):
        chunk_score = 0.0
        for e in ents:
            e_lower = e.lower()
            # check if any expanded term matches
            for term, weight in expanded_terms.items():
                if term in e_lower or e_lower in term:
                    chunk_score += weight
                    break  # mỗi entity chỉ count 1 lần
        scores[i] = chunk_score
    
    # normalize
    if scores.max() > 0:
        scores = scores / (scores.max() + 1e-12)
    
    return scores


def kg_facts_from_expanded(expanded_terms: Dict[str, float], limit: int = 12) -> List[str]:
    """
    Lấy facts từ KG dựa trên expanded terms, ưu tiên high-weight terms.
    """
    facts = []
    seen_edges = set()
    
    # sort terms by weight
    sorted_terms = sorted(expanded_terms.items(), key=lambda x: x[1], reverse=True)
    
    for term, weight in sorted_terms[:10]:
        if len(facts) >= limit:
            break
        
        matching_nodes = [n for n in KG.nodes() if term in str(n).lower()]
        
        for node in matching_nodes[:2]:
            for nb in list(KG.successors(node))[:3]:
                edge_key = (str(node), str(nb))
                if edge_key in seen_edges:
                    continue
                seen_edges.add(edge_key)
                
                edge_data = KG.get_edge_data(node, nb) or {}
                rel = edge_data.get("relation", "related_to")
                edge_weight = edge_data.get("weight", 1)
                
                # chỉ lấy edges có weight >= 2 (co-occur nhiều lần)
                if edge_weight >= 2:
                    facts.append(f"{node} --{rel}[w={edge_weight}]--> {nb}")
                
                if len(facts) >= limit:
                    return facts
    
    return facts


def retrieve_context_graphrag(query: str, top_k: int = TOP_K, alpha: float = ALPHA, 
                               kg_hops: int = 2, kg_weight: float = 0.3) -> Tuple[str, List[int], Dict]:
    """
    GraphRAG retrieval với KG expansion.
    
    Pipeline:
    1. Extract seed terms từ query
    2. Expand terms qua KG (multi-hop traversal)
    3. Semantic search (FAISS)
    4. Graph-based scoring với expanded terms
    5. Hybrid rerank: alpha*semantic + (1-alpha)*graph
    6. Inject KG facts vào context
    
    Returns: (context_str, chunk_indices, debug_info)
    """
    # Step 1: seed terms
    seed_terms = fast_terms(query)
    
    # Step 2: KG expansion
    expanded_terms = get_kg_expanded_terms(seed_terms, hops=kg_hops)
    
    # Step 3: semantic search
    sem_results = semantic_search(query, top_k=top_k * 3)  # lấy nhiều hơn để rerank
    
    # Step 4: graph scoring với expanded terms
    gs = graph_scores_v2(expanded_terms)
    
    # Step 5: hybrid rerank
    combined = []
    for idx, sem_score in sem_results:
        g_score = float(gs[idx])
        # weighted combination
        final_score = alpha * sem_score + (1 - alpha) * g_score
        combined.append((idx, final_score, sem_score, g_score))
    
    combined.sort(key=lambda x: x[1], reverse=True)
    top_chunks = combined[:top_k]
    idxs = [i for i, _, _, _ in top_chunks]
    
    # Step 6: build context
    ctx_parts = [DATASET[i] for i in idxs]
    ctx = "\n\n---\n\n".join(ctx_parts)
    
    # KG facts từ expanded terms (filtered by weight)
    facts = kg_facts_from_expanded(expanded_terms, limit=10)
    if facts:
        ctx += "\n\n===KG_FACTS===\n" + "\n".join(facts)
    
    # debug info
    debug = {
        "seed_terms": list(seed_terms),
        "expanded_terms_count": len(expanded_terms),
        "top_expanded": sorted(expanded_terms.items(), key=lambda x: x[1], reverse=True)[:15],
        "chunk_scores": [(i, f"sem={s:.3f} graph={g:.3f} final={f:.3f}") 
                         for i, f, s, g in top_chunks]
    }
    
    return ctx, idxs, debug


def ask_graphrag(query: str, verbose: bool = False):
    """
    GraphRAG-powered Q&A.
    """
    ctx, idxs, debug = retrieve_context_graphrag(query)
    
    prompt = f"""Bạn là trợ lý hỏi đáp luật Việt Nam.

Bạn là trợ lý hỏi đáp luật Việt Nam.

sử dụng những kiến thức từ knowledge graph, không tự bịa đặt nhé

hơn nữa, những kiến thức có trong CONTEXT rất quan trọng, phải dùng đến chúng để trả lời câu hỏi. Nên hãy dùng hết để tóm tắt để trả lời

CHỈ trả lời dựa trên CONTEXT. Không bịa.
Nếu CONTEXT không có thông tin, trả lời đúng 1 dòng:
no relevant information found

CONTEXT:
{ctx}

CÂU HỎI:
{query}

TRẢ LỜI:"""

    ans = llm_answer(prompt, temperature=0.2, max_tokens=600)
    
    # if verbose:
    #     print("\n" + "="*60)
    #     print("🔍 DEBUG INFO:")
    #     print(f"   Seed terms: {debug['seed_terms']}")
    #     print(f"   Expanded terms count: {debug['expanded_terms_count']}")
    #     print(f"   Top expanded: {[t for t,_ in debug['top_expanded'][:8]]}")
    #     print("   Chunk scores:")
    #     for chunk_id, score_str in debug['chunk_scores']:
    #         print(f"      Chunk {chunk_id}: {score_str}")
    #     print("="*60 + "\n")
    
    return ans, idxs, debug


# ============================================================
# So sánh v1 (cũ) vs v2 (GraphRAG)
# ============================================================

def compare_retrieval(query: str):
    """So sánh kết quả giữa retrieve_context (v1) và retrieve_context_graphrag (v2)"""
    
    print("="*70)
    print(f"QUERY: {query}")
    print("="*70)
    
    # V1 - Original
    ctx_v1, idxs_v1 = retrieve_context(query)
    
    # V2 - GraphRAG
    ctx_v2, idxs_v2, debug = retrieve_context_graphrag(query)
    
    print("\n📌 V1 (Original RAG) - Chunks retrieved:", idxs_v1)
    print("📌 V2 (GraphRAG)     - Chunks retrieved:", idxs_v2)
    
    # Overlap
    overlap = set(idxs_v1) & set(idxs_v2)
    only_v1 = set(idxs_v1) - set(idxs_v2)
    only_v2 = set(idxs_v2) - set(idxs_v1)
    
    print(f"\n🔄 Overlap: {len(overlap)} chunks - {overlap}")
    print(f"➕ Only in V1: {only_v1}")
    print(f"➕ Only in V2 (từ KG expansion): {only_v2}")
    
    print(f"\n🧠 KG Expansion:")
    print(f"   Seed terms: {debug['seed_terms']}")
    print(f"   Expanded to: {debug['expanded_terms_count']} terms")
    print(f"   Top expanded: {[f'{t}({w:.2f})' for t,w in debug['top_expanded'][:10]]}")
    
    return idxs_v1, idxs_v2, debug


# ============================================================
# TEST
# ============================================================

print("✅ GraphRAG Engine v2 loaded!")
print("   - get_kg_expanded_terms(): KG traversal để expand query terms")
print("   - graph_scores_v2(): weighted scoring với expanded terms")
print("   - retrieve_context_graphrag(): full GraphRAG pipeline")
print("   - ask_graphrag(): Q&A với GraphRAG")
print("   - compare_retrieval(): so sánh v1 vs v2")
print()

# Quick test
test_question = "Thuế giá trị gia tăng (GTGT) được định nghĩa như thế nào"
print("🧪 Testing với câu hỏi:", test_question)
print()

# answer, chunks, debug = ask_graphrag(test_question, verbose=True)
answer, chunks, debug = ask_graphrag(test_question, verbose=True)
print("📝 ANSWER:")
print(answer)

✅ GraphRAG Engine v2 loaded!
   - get_kg_expanded_terms(): KG traversal để expand query terms
   - graph_scores_v2(): weighted scoring với expanded terms
   - retrieve_context_graphrag(): full GraphRAG pipeline
   - ask_graphrag(): Q&A với GraphRAG
   - compare_retrieval(): so sánh v1 vs v2

🧪 Testing với câu hỏi: Thuế giá trị gia tăng (GTGT) được định nghĩa như thế nào



Batches: 100%|██████████| 1/1 [00:00<00:00, 39.90it/s]
INFO:httpx:HTTP Request: POST https://integrate.api.nvidia.com/v1/chat/completions "HTTP/1.1 200 OK"


📝 ANSWER:
no relevant information found


In [34]:
# Cell 9: GraphRAG Engine v2 - Full Implementation (FIXED)

from typing import Dict, List, Set, Tuple

# ============================================================
# GRAPHRAG ENGINE V2 - với KG Expansion cho Retrieval
# ============================================================

def get_kg_expanded_terms(seed_terms: Set[str], hops: int = 2, max_neighbors_per_node: int = 5) -> Dict[str, float]:
    """
    Từ seed_terms, đi qua KG để lấy thêm related terms với decay weight.
    Returns: {term: weight} — weight càng cao = càng relevant
    """
    expanded = {}
    
    # seed terms có weight = 1.0
    for t in seed_terms:
        expanded[t.lower()] = 1.0
    
    # BFS expansion với decay
    frontier = list(seed_terms)
    visited = set(t.lower() for t in seed_terms)
    
    for hop in range(1, hops + 1):
        decay = 1.0 / (hop + 1)  # hop 1 → 0.5, hop 2 → 0.33
        next_frontier = []
        
        for term in frontier:
            # tìm nodes trong KG match term
            matching_nodes = [n for n in KG.nodes() if term.lower() in str(n).lower()]
            
            for node in matching_nodes[:3]:  # giới hạn nodes per term
                # lấy neighbors (cả successors và predecessors)
                neighbors = list(KG.successors(node)) + list(KG.predecessors(node))
                
                # sort by edge weight nếu có
                weighted_neighbors = []
                for nb in neighbors:
                    edge_data = KG.get_edge_data(node, nb) or KG.get_edge_data(nb, node) or {}
                    w = edge_data.get("weight", 1)
                    weighted_neighbors.append((nb, w))
                
                weighted_neighbors.sort(key=lambda x: x[1], reverse=True)
                
                for nb, edge_weight in weighted_neighbors[:max_neighbors_per_node]:
                    nb_lower = str(nb).lower()
                    if nb_lower not in visited:
                        visited.add(nb_lower)
                        next_frontier.append(str(nb))
                        # weight = decay * normalized edge weight
                        score = decay * min(edge_weight / 5.0, 1.0)
                        expanded[nb_lower] = max(expanded.get(nb_lower, 0), score)
        
        frontier = next_frontier
    
    return expanded


def graph_scores_v2(expanded_terms: Dict[str, float]) -> np.ndarray:
    """
    Chấm điểm chunks dựa trên expanded terms từ KG traversal.
    Weighted matching thay vì binary.
    """
    scores = np.zeros(len(DATASET), dtype=float)
    if not expanded_terms:
        return scores
    
    for i, ents in enumerate(CHUNK_ENTITIES):
        chunk_score = 0.0
        for e in ents:
            e_lower = e.lower()
            # check if any expanded term matches
            for term, weight in expanded_terms.items():
                if term in e_lower or e_lower in term:
                    chunk_score += weight
                    break  # mỗi entity chỉ count 1 lần
        scores[i] = chunk_score
    
    # normalize
    if scores.max() > 0:
        scores = scores / (scores.max() + 1e-12)
    
    return scores


def kg_facts_from_expanded(expanded_terms: Dict[str, float], limit: int = 12) -> List[str]:
    """
    Lấy facts từ KG dựa trên expanded terms, ưu tiên high-weight terms.
    """
    facts = []
    seen_edges = set()
    
    # sort terms by weight
    sorted_terms = sorted(expanded_terms.items(), key=lambda x: x[1], reverse=True)
    
    for term, weight in sorted_terms[:10]:
        if len(facts) >= limit:
            break
        
        matching_nodes = [n for n in KG.nodes() if term in str(n).lower()]
        
        for node in matching_nodes[:2]:
            for nb in list(KG.successors(node))[:3]:
                edge_key = (str(node), str(nb))
                if edge_key in seen_edges:
                    continue
                seen_edges.add(edge_key)
                
                edge_data = KG.get_edge_data(node, nb) or {}
                rel = edge_data.get("relation", "related_to")
                edge_weight = edge_data.get("weight", 1)
                
                # Lấy tất cả edges (bỏ filter weight >= 2)
                facts.append(f"{node} --{rel}[w={edge_weight}]--> {nb}")
                
                if len(facts) >= limit:
                    return facts
    
    return facts


def retrieve_context_graphrag(query: str, top_k: int = TOP_K, alpha: float = ALPHA, 
                               kg_hops: int = 2) -> Tuple[str, List[int], Dict]:
    """
    GraphRAG retrieval với KG expansion.
    
    Pipeline:
    1. Extract seed terms từ query
    2. Expand terms qua KG (multi-hop traversal)
    3. Semantic search (FAISS)
    4. Graph-based scoring với expanded terms
    5. Hybrid rerank: alpha*semantic + (1-alpha)*graph
    6. Inject KG facts vào context
    
    Returns: (context_str, chunk_indices, debug_info)
    """
    # Step 1: seed terms
    seed_terms = fast_terms(query)
    
    # Step 2: KG expansion
    expanded_terms = get_kg_expanded_terms(seed_terms, hops=kg_hops)
    
    # Step 3: semantic search - LẤY NHIỀU HƠN để có pool rerank
    sem_results = semantic_search(query, top_k=top_k * 3)
    
    # Step 4: graph scoring với expanded terms
    gs = graph_scores_v2(expanded_terms)
    
    # Step 5: hybrid rerank
    combined = []
    for idx, sem_score in sem_results:
        g_score = float(gs[idx])
        # weighted combination
        final_score = alpha * sem_score + (1 - alpha) * g_score
        combined.append((idx, final_score, sem_score, g_score))
    
    combined.sort(key=lambda x: x[1], reverse=True)
    top_chunks = combined[:top_k]
    idxs = [i for i, _, _, _ in top_chunks]
    
    # Step 6: build context - GIỐNG V1: dùng "\n\n" thay vì "---"
    ctx_parts = [DATASET[i] for i in idxs]
    ctx = "\n\n".join(ctx_parts)
    
    # KG facts từ expanded terms
    facts = kg_facts_from_expanded(expanded_terms, limit=10)
    if facts:
        ctx += "\n\nKG_FACTS:\n" + "\n".join(facts)
    
    # debug info
    debug = {
        "seed_terms": list(seed_terms),
        "expanded_terms_count": len(expanded_terms),
        "top_expanded": sorted(expanded_terms.items(), key=lambda x: x[1], reverse=True)[:15],
        "chunk_scores": [(i, f"sem={s:.3f} graph={g:.3f} final={f:.3f}") 
                         for i, f, s, g in top_chunks]
    }
    
    return ctx, idxs, debug


def ask_graphrag(query: str, verbose: bool = False):
    """
    GraphRAG-powered Q&A.
    """
    ctx, idxs, debug = retrieve_context_graphrag(query)
    
    # PROMPT GIỐNG V1
    prompt = f"""Bạn là trợ lý hỏi đáp luật Việt Nam.

sử dụng những kiến thức từ knowledge graph, không tự bịa đặt nhé

hơn nữa, những kiến thức có trong CONTEXT rất quan trọng, phải dùng đến chúng để trả lời câu hỏi. Nên hãy dùng hết để tóm tắt để trả lời

CHỈ trả lời dựa trên CONTEXT. Không bịa.
Nếu CONTEXT không có thông tin, trả lời đúng 1 dòng:
no relevant information found

CONTEXT:
{ctx}

QUESTION:
{query}

ANSWER:
"""

    ans = llm_answer(prompt, temperature=0.2, max_tokens=600)
    
    if verbose:
        print("\n" + "="*60)
        print("🔍 DEBUG INFO:")
        print(f"   Seed terms: {debug['seed_terms']}")
        print(f"   Expanded terms count: {debug['expanded_terms_count']}")
        print(f"   Top expanded: {[t for t,_ in debug['top_expanded'][:8]]}")
        print("   Chunk scores:")
        for chunk_id, score_str in debug['chunk_scores']:
            print(f"      Chunk {chunk_id}: {score_str}")
        print("="*60 + "\n")
    
    return ans, idxs, debug


# ============================================================
# So sánh v1 (cũ) vs v2 (GraphRAG)
# ============================================================

def compare_retrieval(query: str):
    """So sánh kết quả giữa retrieve_context (v1) và retrieve_context_graphrag (v2)"""
    
    print("="*70)
    print(f"QUERY: {query}")
    print("="*70)
    
    # V1 - Original
    ctx_v1, idxs_v1 = retrieve_context(query)
    
    # V2 - GraphRAG
    ctx_v2, idxs_v2, debug = retrieve_context_graphrag(query)
    
    print("\n📌 V1 (Original RAG) - Chunks retrieved:", idxs_v1)
    print("📌 V2 (GraphRAG)     - Chunks retrieved:", idxs_v2)
    
    # Overlap
    overlap = set(idxs_v1) & set(idxs_v2)
    only_v1 = set(idxs_v1) - set(idxs_v2)
    only_v2 = set(idxs_v2) - set(idxs_v1)
    
    print(f"\n🔄 Overlap: {len(overlap)} chunks - {overlap}")
    print(f"➕ Only in V1: {only_v1}")
    print(f"➕ Only in V2 (từ KG expansion): {only_v2}")
    
    print(f"\n🧠 KG Expansion:")
    print(f"   Seed terms: {debug['seed_terms']}")
    print(f"   Expanded to: {debug['expanded_terms_count']} terms")
    print(f"   Top expanded: {[f'{t}({w:.2f})' for t,w in debug['top_expanded'][:10]]}")
    
    # So sánh context
    print(f"\n📄 Context lengths: V1={len(ctx_v1)} chars | V2={len(ctx_v2)} chars")
    
    return idxs_v1, idxs_v2, debug


# ============================================================
# TEST
# ============================================================

print("✅ GraphRAG Engine v2 loaded!")
print("   - get_kg_expanded_terms(): KG traversal để expand query terms")
print("   - graph_scores_v2(): weighted scoring với expanded terms")
print("   - retrieve_context_graphrag(): full GraphRAG pipeline")
print("   - ask_graphrag(): Q&A với GraphRAG")
print("   - compare_retrieval(): so sánh v1 vs v2")
print()

# Quick test
test_question = "Thuế giá trị gia tăng (GTGT) được định nghĩa như thế nào?"
print("🧪 Testing với câu hỏi:", test_question)
print()

# So sánh V1 vs V2 trước
print("📊 SO SÁNH V1 vs V2:")
compare_retrieval(test_question)

print("\n" + "="*70)
print("📝 GRAPHRAG ANSWER:")
print("="*70)
answer, chunks, debug = ask_graphrag(test_question, verbose=True)
print(answer)

✅ GraphRAG Engine v2 loaded!
   - get_kg_expanded_terms(): KG traversal để expand query terms
   - graph_scores_v2(): weighted scoring với expanded terms
   - retrieve_context_graphrag(): full GraphRAG pipeline
   - ask_graphrag(): Q&A với GraphRAG
   - compare_retrieval(): so sánh v1 vs v2

🧪 Testing với câu hỏi: Thuế giá trị gia tăng (GTGT) được định nghĩa như thế nào?

📊 SO SÁNH V1 vs V2:
QUERY: Thuế giá trị gia tăng (GTGT) được định nghĩa như thế nào?


Batches: 100%|██████████| 1/1 [00:00<00:00, 38.00it/s]



📌 V1 (Original RAG) - Chunks retrieved: [2332, 2334, 2371, 2400, 2418, 2374]
📌 V2 (GraphRAG)     - Chunks retrieved: [2371, 1181, 2331, 2418, 2360, 2374]

🔄 Overlap: 3 chunks - {2418, 2371, 2374}
➕ Only in V1: {2400, 2332, 2334}
➕ Only in V2 (từ KG expansion): {2360, 2331, 1181}

🧠 KG Expansion:
   Seed terms: ['giá', 'trị', 'thuế', 'tăng', 'gia', 'nghĩa', 'gtgt']
   Expanded to: 190 terms
   Top expanded: ['giá(1.00)', 'trị(1.00)', 'thuế(1.00)', 'tăng(1.00)', 'gia(1.00)', 'nghĩa(1.00)', 'gtgt(1.00)', 'thuế giá(0.50)', 'thuế giá trị(0.50)', 'thuế điều(0.50)']

📄 Context lengths: V1=4186 chars | V2=4238 chars

📝 GRAPHRAG ANSWER:


Batches: 100%|██████████| 1/1 [00:00<00:00, 42.05it/s]
INFO:httpx:HTTP Request: POST https://integrate.api.nvidia.com/v1/chat/completions "HTTP/1.1 200 OK"



🔍 DEBUG INFO:
   Seed terms: ['giá', 'trị', 'thuế', 'tăng', 'gia', 'nghĩa', 'gtgt']
   Expanded terms count: 190
   Top expanded: ['giá', 'trị', 'thuế', 'tăng', 'gia', 'nghĩa', 'gtgt', 'thuế giá']
   Chunk scores:
      Chunk 2371: sem=0.849 graph=0.917 final=0.859
      Chunk 1181: sem=0.843 graph=0.935 final=0.857
      Chunk 2331: sem=0.841 graph=0.944 final=0.857
      Chunk 2418: sem=0.849 graph=0.900 final=0.856
      Chunk 2360: sem=0.844 graph=0.796 final=0.837
      Chunk 2374: sem=0.847 graph=0.733 final=0.830

Thuế giá trị gia tăng (GTGT) là loại thuế tiêu dùng được tính trên giá trị tăng thêm của hàng hoá, dịch vụ phát sinh trong quá trình sản xuất, lưu thông đến tiêu dùng; căn cứ tính thuế gồm giá tính thuế và thuế suất (Điều 6 Luật Thuế GTGT 2008).


In [30]:
# question = "Hoạt động thương mại điện tử được hiểu như thế nào?"
question = "Thuế giá trị gia tăng (GTGT) được định nghĩa như thế nào?"

context, used_chunks = retrieve_context(question)

print("QUESTION:")
print(question)

print("\nCONTEXT USED:")
print(context)

print("\nANSWER:")
print(llm_answer(f"""
Bạn là trợ lý hỏi đáp luật Việt Nam.

CHỈ trả lời dựa trên CONTEXT.
Nếu không có thông tin, trả lời:
no relevant information found

CONTEXT:
{context}

QUESTION:
{question}

ANSWER:
"""))

print("\nUSED CHUNKS:")
print(used_chunks)


Batches: 100%|██████████| 1/1 [00:00<00:00, 35.86it/s]


QUESTION:
Thuế giá trị gia tăng (GTGT) được định nghĩa như thế nào?

CONTEXT USED:
[article] Văn bản: Luật thuế giá trị gia tăng 2008 số 13/2008/QH12 áp dụng 2024 > Nguồn: GTGT2008.json > URL: https://thuvienphapluat.vn/van-ban/Thue-Phi-Le-Phi/Luat-thue-gia-tri-gia-tang-2008-13-2008-QH12-66934.aspx > Crawled: 2026-01-24 > Chương 1.
NHỮNG QUY ĐỊNH CHUNG > Điều 2. Thuế giá trị gia tăng
Thuế giá trị gia tăng là thuế
tính trên giá trị tăng thêm của hàng hóa, dịch vụ phát sinh trong quá trình từ
sản xuất, lưu thông đến tiêu dùng.

[article] Văn bản: Luật thuế giá trị gia tăng 2008 số 13/2008/QH12 áp dụng 2024 > Nguồn: GTGT2008.json > URL: https://thuvienphapluat.vn/van-ban/Thue-Phi-Le-Phi/Luat-thue-gia-tri-gia-tang-2008-13-2008-QH12-66934.aspx > Crawled: 2026-01-24 > Chương 1.
NHỮNG QUY ĐỊNH CHUNG > Điều 4. Người nộp thuế
Người nộp thuế giá trị gia tăng
là tổ chức, cá nhân sản xuất, kinh doanh hàng hóa, dịch vụ chịu thuế giá trị
gia tăng (sau đây gọi là cơ sở kinh doanh) và tổ chức, cá nhân

INFO:httpx:HTTP Request: POST https://integrate.api.nvidia.com/v1/chat/completions "HTTP/1.1 200 OK"


Thuế giá trị gia tăng là thuế tính trên giá trị tăng thêm của hàng hóa, dịch vụ phát sinh trong quá trình từ sản xuất, lưu thông đến tiêu dùng.

USED CHUNKS:
[2332, 2334, 2371, 2400, 2418, 2374]
